# 《安娜卡列尼娜》新编——利用TensorFlow构建LSTM模型

最近看完了LSTM的一些外文资料，主要参考了Colah的blog以及Andrej Karpathy blog的一些关于RNN的材料，准备动手去实现一个LSTM模型。代码的基础框架来自于Udacity上深度学习纳米学位的课程（付费课程）的一个demo，我刚开始看代码的时候真的是一头雾水，很多东西没有理解，后来反复查阅资料，并我重新对代码进行了学习和修改，对步骤进行了进一步的剖析，下面将一步步用TensorFlow来构建LSTM模型进行文本学习并试图去生成新的文本。

关于RNN与LSTM模型本文不做介绍，详情去查阅资料过着去看上面的blog链接，讲的很清楚啦。这篇文章主要是偏向实战，来自己动手构建LSTM模型。

数据集来自于外文版《安娜卡列妮娜》书籍的文本文档（本文后面会提供整个project的git链接）。

In [19]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf


# 解决 Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 这个问题
import os  
# os.environ["TF_CPP_MIN_LOG_LEVEL"]='1' # 这是默认的显示等级，显示所有信息  
os.environ["TF_CPP_MIN_LOG_LEVEL"]='2' # 只显示 warning 和 Error   
# os.environ["TF_CPP_MIN_LOG_LEVEL"]='3' # 只显示 Error

# 1 数据加载与预处理

In [4]:
with open('data/anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [5]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [6]:
encoded[:100]

array([30, 72, 81, 80, 26, 27, 57, 41, 65, 52, 52, 52,  0, 81, 80, 80,  2,
       41,  1, 81,  7, 68, 34, 68, 27, 60, 41, 81, 57, 27, 41, 81, 34, 34,
       41, 81, 34, 68, 24, 27, 31, 41, 27, 28, 27, 57,  2, 41, 36, 13, 72,
       81, 80, 80,  2, 41,  1, 81,  7, 68, 34,  2, 41, 68, 60, 41, 36, 13,
       72, 81, 80, 80,  2, 41, 68, 13, 41, 68, 26, 60, 41, 10, 69, 13, 52,
       69, 81,  2, 37, 52, 52, 77, 28, 27, 57,  2, 26, 72, 68, 13])

In [7]:
len(vocab)

83

# 2 分割mini-batch



<img src="assets/sequence_batching@1x.png" width=500px>


完成了前面的数据预处理操作，接下来就是要划分我们的数据集，在这里我们使用mini-batch来进行模型训练，那么我们要如何划分数据集呢？在进行mini-batch划分之前，我们先来了解几个概念。

假如我们目前手里有一个序列1-12，我们接下来以这个序列为例来说明划分mini-batch中的几个概念。首先我们回顾一下，在DNN和CNN中，我们都会将数据分batch输入给神经网络，加入我们有100个样本，如果设置我们的batch_size=10，那么意味着每次我们都会向神经网络输入10个样本进行训练调整参数。同样的，在LSTM中，batch_size意味着每次向网络输入多少个样本，在上图中，当我们设置batch_size=2时，我们会将整个序列划分为6个batch，每个batch中有两个数字。

然而由于RNN中存在着“记忆”，也就是循环。事实上一个循环神经网络能够被看做是多个相同神经网络的叠加，在这个系统中，每一个网络都会传递信息给下一个。上面的图中，我们可以看到整个RNN网络由三个相同的神经网络单元叠加起来的序列。那么在这里就有了第二个概念sequence_length（也叫steps），中文叫序列长度。上图中序列长度是3，可以看到将三个字符作为了一个序列。

有了上面两个概念，我们来规范一下后面的定义。我们定义一个batch中的序列个数为N（batch_size），定义单个序列长度为M（也就是我们的steps）。那么实际上我们每个batch是一个N x M的数组。在这里我们重新定义batch_size为一个N x M的数组，而不是batch中序列的个数。在上图中，当我们设置N=2， M=3时，我们可以得到每个batch的大小为2 x 3 = 6个字符，整个序列可以被分割成12 / 6 = 2个batch。

In [8]:
def get_batches(arr, n_seqs, n_steps):
    '''
    对已有的数组进行mini-batch分割
    
    arr: 待分割的数组
    n_seqs: 一个batch中序列个数
    n_steps: 单个序列包含的字符数
    '''
    
    batch_size = n_seqs * n_steps
    n_batches = int(len(arr) / batch_size)
    # 这里我们仅保留完整的batch，对于不能整出的部分进行舍弃
    arr = arr[:batch_size * n_batches]
    
    # 重塑
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # inputs
        x = arr[:, n:n+n_steps]
        # targets
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

上面的代码定义了一个generator，调用函数会返回一个generator对象，我们可以获取一个batch。

In [9]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [10]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[30 72 81 80 26 27 57 41 65 52]
 [41 81  7 41 13 10 26 41 74 10]
 [28 68 13 37 52 52 40 66 27 60]
 [13 41 38 36 57 68 13 74 41 72]
 [41 68 26 41 68 60 53 41 60 68]
 [41 42 26 41 69 81 60 52 10 13]
 [72 27 13 41 20 10  7 27 41  1]
 [31 41 61 36 26 41 13 10 69 41]
 [26 41 68 60 13 71 26 37 41 25]
 [41 60 81 68 38 41 26 10 41 72]]

y
 [[72 81 80 26 27 57 41 65 52 52]
 [81  7 41 13 10 26 41 74 10 68]
 [68 13 37 52 52 40 66 27 60 53]
 [41 38 36 57 68 13 74 41 72 68]
 [68 26 41 68 60 53 41 60 68 57]
 [42 26 41 69 81 60 52 10 13 34]
 [27 13 41 20 10  7 27 41  1 10]
 [41 61 36 26 41 13 10 69 41 60]
 [41 68 60 13 71 26 37 41 25 72]
 [60 81 68 38 41 26 10 41 72 27]]


# 3 模型构建
模型构建部分主要包括了输入层，LSTM层，输出层，loss，optimizer等部分的构建，我们将一块一块来进行实现。

## 3.1 输入层

In [11]:
def build_inputs(num_seqs, num_steps):
    '''
    构建输入层
    
    num_seqs: 每个batch中的序列个数
    num_steps: 每个序列包含的字符数
    '''
    inputs = tf.placeholder(tf.int32, shape=(num_seqs, num_steps), name='inputs')
    targets = tf.placeholder(tf.int32, shape=(num_seqs, num_steps), name='targets')
    
    # 加入keep_prob
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

## 3.2 LSTM层

In [12]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' 
    构建lstm层
        
    keep_prob
    lstm_size: lstm隐层中结点数目
    num_layers: lstm的隐层数目
    batch_size: batch_size

    '''  
    
    cells=[]
    for n in range(num_layers):
        cells.append(tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.BasicLSTMCell(lstm_size), output_keep_prob=keep_prob))
    cell = tf.nn.rnn_cell.MultiRNNCell(cells)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state


## 3.3 输出层

In [13]:
def build_output(lstm_output, in_size, out_size):
    ''' 
    构造输出层
        
    lstm_output: lstm层的输出结果
    in_size: lstm输出层重塑后的size
    out_size: softmax层的size
    
    '''

    # 将lstm的输出按照列concate，例如[[1,2,3],[7,8,9]],
    # tf.concat的结果是[1,2,3,7,8,9]
    seq_output = tf.concat(lstm_output,axis=1) # tf.concat(concat_dim, values)
    # reshape
    x = tf.reshape(seq_output, [-1, in_size])
    
    # 将lstm层与softmax层全连接
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal([in_size, out_size], stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # 计算logits
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # softmax层返回概率分布
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

## 3.4 训练误差计算

In [14]:
def build_loss(logits, targets, lstm_size, num_classes):
    '''
    根据logits和targets计算损失
    
    logits: 全连接层的输出结果（不经过softmax）
    targets: targets
    lstm_size
    num_classes: vocab_size
        
    '''
    
    # One-hot编码
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    
    return loss

## 3.5 Optimizer
我们知道RNN会遇到梯度爆炸（gradients exploding）和梯度弥散（gradients disappearing)的问题。LSTM解决了梯度弥散的问题，但是gradient仍然可能会爆炸，因此我们采用gradient clippling的方式来防止梯度爆炸。即通过设置一个阈值，当gradients超过这个阈值时，就将它重置为阈值大小，这就保证了梯度不会变得很大。

In [15]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' 
    构造Optimizer
   
    loss: 损失
    learning_rate: 学习率
    
    '''
    
    # 使用clipping gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

## 3.6 模型组合
使用tf.nn.dynamic_run来运行RNN序列

In [16]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # 如果sampling是True，则采用SGD
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # 输入层
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # LSTM层
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        # 对输入进行one-hot编码
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # 运行RNN
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # 预测结果
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss 和 optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

# 4 模型训练

### 参数设置
在模型训练之前，我们首先初始化一些参数，我们的参数主要有：

- num_seqs: 单个batch中序列的个数
- num_steps: 单个序列中字符数目
- lstm_size: 隐层结点个数
- num_layers: LSTM层个数
- learning_rate: 学习率
- keep_prob: dropout层中保留结点比例

In [17]:
batch_size = 100         # Sequences per batch
num_steps = 100          # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001    # Learning rate
keep_prob = 0.5         # Dropout keep probability

In [21]:
epochs = 20
# 每n轮进行一次变量保存
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(tf.global_variables_initializer())
    
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            # control the print lines
            if counter % 100 == 0:
                print('轮数: {}/{}... '.format(e+1, epochs),
                      '训练步数: {}... '.format(counter),
                      '训练误差: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))

            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

KeyboardInterrupt: 

In [ ]:
# 查看checkpoints
tf.train.get_checkpoint_state('checkpoints')

# 5 文本生成
现在我们可以基于我们的训练参数进行文本的生成。当我们输入一个字符时，LSTM会预测下一个字符，我们再将新的字符进行输入，这样能不断的循环下去生成本文。

为了减少噪音，每次的预测值我会选择最可能的前5个进行随机选择，比如输入h，预测结果概率最大的前五个为[o,e,i,u,b]，我们将随机从这五个中挑选一个作为新的字符，让过程加入随机因素会减少一些噪音的生成。

In [18]:
def pick_top_n(preds, vocab_size, top_n=5):
    """
    从预测结果中选取前top_n个最可能的字符
    
    preds: 预测结果
    vocab_size
    top_n
    """
    p = np.squeeze(preds)
    # 将除了top_n个预测值的位置都置为0
    p[np.argsort(p)[:-top_n]] = 0
    # 归一化概率
    p = p / np.sum(p)
    # 随机选取一个字符
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [19]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    """
    生成新文本
    
    checkpoint: 某一轮迭代的参数文件
    n_sample: 新闻本的字符长度
    lstm_size: 隐层结点数
    vocab_size
    prime: 起始文本
    """
    # 将输入的单词转换为单个字符组成的list
    samples = [c for c in prime]
    # sampling=True意味着batch的size=1 x 1
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        # 加载模型参数，恢复训练
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            # 输入单个字符
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        # 添加字符到samples中
        samples.append(int_to_vocab[c])
        
        # 不断生成字符，直到达到指定数目
        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [20]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i3960_l512.ckpt'

In [26]:
# 选用最终的训练参数作为输入进行文本生成
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="The")
print(samp)

Ther woold have saw that it was a second of a little.

"The princess went a presence out of the music. Then he'd to be dropped. And
I'm not saigh."

"Well, and he was not a man. That's which I have been troubled to supposing
her transformed, and his woman, that here is not a shartes of hesitation."

"Well, that you will not see you all to the delight of all he he didn't
believe it."

"What do you think? I shall see her, and he has not asked it in, and I could
call me a little bride."

"What a man of the same in that, I'll go to him," said Alexey
Alexandrovitch, smiling to himself, "they will be angry, to say. The
contrary is an inside. It's nine thinging to him in argive, but I
don't want to tell you, and I didn't come," he said, and he found to
get the presence of side, he was not too stopped in her eyes.

An halves, both, though these pares, he heard the seat of the charmage, watching
the pather, and he was seen in the poor the things of a peculiar love,
his feech one sigher she had 

In [22]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farsd othas sead ansisg he therinn wen an time he has tas hers wand so the her ansong astens hit
ang arideret to hess the
cand the wan ou he and waste sad that had whe hh simorsersond torith we sotered.

"Whot simte wet ant ans wit hise afde sit hat her touten th merint he tor he chins.

"ho se ar her ore tir that hes hor
she there tothe werilth se ses ororing, thas te that ant he ser ot hit he corlithit."
"Sele, Ir anthe wast to han sist hos the coseling thet tir sassesesin thon the tan sheres an tor the ceras tat aless the
hans, wins of the hite ho tar so at ane an to tont his the whers and
ther herasdith shere th th woul ter hor
the he wothe hires te sel hin hind and the he sotor has has andeses onet toren his of athand this thar sost the soud
ale ar on hot orin at the her ses on the sesed tor het
an ase thing, at hins hat had hhe was shares thin and his ofe the hor th the sised ther
alind tor here tor ther thar

one tanes, she thes sotithe with toused tat her aret al hans
on he he 

In [23]:
checkpoint = 'checkpoints/i1000_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farrer, as his positions
the child of happeness and
she wither she wan sainted at the petsaot, that his expression of his had, but he wanted to an the drien that whome he haved to the pertion with the count and that the perpleation of the mungrait of the cander were sate along the same at
her all this he heard the
stallers, and that she saw she same and shade all with him he had not said to stelp only with the sorm a misters
one when he could not he
stought."

"Yes, and
you sand to ser anster and som that that?
Ind to be a candies, and shilly see that with the down of the semilial all or
taking,
and thought he said staid at the clate offen as had almoment to be anouted happy... The presting he deen the conder and had
not so mo that he can't an in a such walked in it a pruce to him.

"I say that have been the seming of the posstor," talking and her thought one
standing his explaning of a that this something worls. So he was stept, and that that when she had been that he wonder to be the

In [24]:
checkpoint = 'checkpoints/i2000_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Fardievna, and had as he children his
hand.

"And his bight me that's she do it's being became string."

"Oh, that, you know where your home is never mented and to do?"

"I could have seen her house and a minute and the socies, that's not as
it's nothing to see in that arest...." And the count of the
sers of tense.

"Well that is she something was that's, the same impersaite with you."

"I'll delighted her?" he added.

"Yes, I can't speak of him only a place of the made in the
raise.

The compitions was in she said if, and still a group of the prince. "That's the higher of
the pashed or to her. What as seemed is in thes though, in a cerrain with
him. His fingers of the middare of the mare they and she had
spenk a land."

"Alexey Alexandrovitch who wrote a going, in her mother, in the
prace was to ble and they always begin to talk and say to
him, and would be the secarisial missed to breather. I'm said: "It's
nothing as a look to hall that I shall go and will see that sick of
him the he